In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
import requests
from datetime import datetime
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [26]:
# Đọc file CSV
# file_path = '/kaggle/input/omdb-links/links.csv'
file_path = '/kaggle/input/omdb-links/test_links.csv'
df = pd.read_csv(file_path)
df

,imdbId
0,1
1,22
2,172
3,1207
4,18806
5,114709


In [27]:
df['imdbId'] = df['imdbId'].apply(lambda x: f'tt{str(x).zfill(7)}')
df

,imdbId
0,tt0000001
1,tt0000022
2,tt0000172
3,tt0001207
4,tt0018806
5,tt0114709


In [28]:
# api_key = '2d918779'
# api_key_list = ['2d918779', '817e9847', '6d85304', 'c05a60d1', '7c616a06', 'f8711aaf', '8e22b5b']
api_key_list = ['7c616a06', 'f8711aaf', '8e22b5b']
# url_template = 'https://www.omdbapi.com/?i={}&apikey=' + api_key

In [29]:
   def get_movie_data(imdb_id, api_key):
    url = f"http://www.omdbapi.com/?i={imdb_id}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()  # Trả về dữ liệu JSON từ OMDB API
    else:
        return None

In [31]:
# Chia DataFrame thành các khối với mỗi khối 1000 dòng
def crawl_movies_by_api_keys(df, api_key_list):
    start = 0
    chunk_size = 2
    for i, api_key in enumerate(api_key_list):
        # Xác định phần của DataFrame dựa trên vị trí
        chunk = df.iloc[start:start + chunk_size]
        
        # Nếu chunk rỗng, nghĩa là hết dữ liệu để crawl
        if chunk.empty:
            break
        
        # Crawl thông tin phim cho từng imdbId trong chunk
        for imdb_id in chunk['imdbId']:
            movie_info = get_movie_data(imdb_id, api_key)
            if movie_info and movie_info.get('Response') == 'True':
                movie_data.append(movie_info)
            time.sleep(1)  # Tránh bị chặn API
        
        # Cập nhật vị trí bắt đầu cho chunk tiếp theo
        start += chunk_size

In [33]:
# Danh sách để lưu thông tin phim
movie_data = []

# Gọi hàm để crawl dữ liệu phim
crawl_movies_by_api_keys(df, api_key_list)

# Chuyển đổi dữ liệu sang DataFrame
df_movies = pd.DataFrame(movie_data)

# Lưu dữ liệu phim vào file CSV mới
output_file = 'movie_data.csv'
df_movies.to_csv(output_file, index=False)

print(f"Đã lưu thông tin phim vào file: {output_file}")

Đã lưu thông tin phim vào file: movie_data.csv


In [ ]:
import pandas as pd

In [6]:
df_raw = pd.read_csv("./movie_data (1).csv")

In [7]:
df_raw

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice
0,Toy Story,1995,G,22-Nov-95,81 min,"Animation, Adventure, Comedy",John Lasseter,"John Lasseter, Pete Docter, Andrew Stanton","Tom Hanks, Tim Allen, Don Rickles",A cowboy doll is profoundly threatened and jea...,...,United States,Nominated for 3 Oscars. 29 wins & 24 nominatio...,https://m.media-amazon.com/images/M/MV5BZTA3OW...,"[{'Source': 'Internet Movie Database', 'Value'...",96.0,8.3,"1,089,101",tt0114709,movie,"$223,225,679"
1,Jumanji,1995,PG,15-Dec-95,104 min,"Adventure, Comedy, Family",Joe Johnston,"Jonathan Hensleigh, Greg Taylor, Jim Strain","Robin Williams, Kirsten Dunst, Bonnie Hunt",When two kids find and play a magical board ga...,...,United States,4 wins & 11 nominations,https://m.media-amazon.com/images/M/MV5BYTFkMj...,"[{'Source': 'Internet Movie Database', 'Value'...",39.0,7.1,"384,663",tt0113497,movie,"$100,499,940"
2,Grumpier Old Men,1995,PG-13,22-Dec-95,101 min,"Comedy, Romance",Howard Deutch,Mark Steven Johnson,"Walter Matthau, Jack Lemmon, Ann-Margret",John and Max resolve to save their beloved bai...,...,United States,2 wins & 2 nominations,https://m.media-amazon.com/images/M/MV5BMDkwYT...,"[{'Source': 'Internet Movie Database', 'Value'...",46.0,6.7,"30,011",tt0113228,movie,"$71,518,503"
3,Waiting to Exhale,1995,R,22-Dec-95,124 min,"Comedy, Drama, Romance",Forest Whitaker,"Terry McMillan, Ron Bass","Whitney Houston, Angela Bassett, Loretta Devine","Based on Terry McMillan's novel, this film fol...",...,United States,9 wins & 10 nominations,https://m.media-amazon.com/images/M/MV5BZWU4Nz...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.0,"12,462",tt0114885,movie,"$67,052,156"
4,Father of the Bride Part II,1995,PG,08-Dec-95,106 min,"Comedy, Family, Romance",Charles Shyer,"Albert Hackett, Frances Goodrich, Nancy Meyers","Steve Martin, Diane Keaton, Martin Short",George Banks must deal not only with his daugh...,...,United States,1 win & 2 nominations,https://m.media-amazon.com/images/M/MV5BOTMwNz...,"[{'Source': 'Internet Movie Database', 'Value'...",49.0,6.1,"42,329",tt0113041,movie,"$76,594,107"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5988,Mother Lode,1982,PG,23-Sep-82,103 min,"Adventure, Mystery, Thriller","Charlton Heston, Fraser C. Heston",Fraser C. Heston,"Charlton Heston, Kim Basinger, Nick Mancuso",It's Silas McGee's gold. He just hasn't found ...,...,United States,NaN,https://m.media-amazon.com/images/M/MV5BMzE2ZG...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,5.7,"1,721",tt0084359,movie,NaN
5989,The Night of the Shooting Stars,1982,R,17-Sep-82,107 min,"Drama, Fantasy, Romance","Paolo Taviani, Vittorio Taviani","Paolo Taviani, Vittorio Taviani, Giuliani G. D...","Omero Antonutti, Margarita Lozano, Claudio Big...","In 1944, the residents of a small Italian town...",...,Italy,19 wins & 13 nominations,https://m.media-amazon.com/images/M/MV5BMGUyYj...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,7.3,"4,649",tt0084422,movie,"$2,257,307"
5990,That Night in Varennes,1982,R,16-Feb-83,150 min,"Drama, History",Ettore Scola,"Sergio Amidei, Ettore Scola, Catherine Rihoit","Jean-Louis Barrault, Marcello Mastroianni, Han...","During the French Revolution, a surprising com...",...,"France, Italy",6 wins & 16 nominations,https://m.media-amazon.com/images/M/MV5BZjkwMm...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,7.2,"2,572",tt0084423,movie,NaN
5991,One from the Heart,1981,R,11-Feb-82,107 min,"Drama, Musical, Romance",Francis Ford Coppola,"Armyan Bernstein, Francis Ford Coppola, Luana ...","Frederic Forrest, Teri Garr, Raul Julia",A couple has a fight after living together 5 y...,...,United States,Nominated for 1 Oscar. 2 nominations total,https://m.media-amazon.com/images/M/MV5BOTliZG...,"[{'Source': 'Internet Movie Database', 'Value'...",57.0,6.5,"7,728",tt0084445,mo